In [1]:
####################MODIFIABLE RUN PARAMETERS#########################    

wellslist=['A1','A2','A3','A4','B1','B2','B3','B4','C1','C2','C3','C4',]
    
tiprack_starting_pos ={
    "tiprack_10" : 'A1',
    "tiprack_300" : 'A1'
}

#Antibody incubation time in seconds
ABS_INCUBATION_TIME = 5400

#debug mode skips all incubations, prints out additional info
debug = False

####################FIXED RUN PARAMETERS######################### 

API_VERSION= '2.7'
default_flow_rate = 50
well_flow_rate = 5
sample_flow_rate = 0.2
wash_volume = 150
USE_TROUGH = True

class Object:
    pass

####################LABWARE LAYOUT ON DECK#########################    
labwarePositions = Object()
labwarePositions.buffers_reservoir = 1
labwarePositions.par2 = 2
labwarePositions.antibodies_plate = 3
labwarePositions.tiprack_300 = 6


In [2]:
####################GENERAL SETUP################################


from opentrons import execute, simulate
import string
import json
import time
import logging
logging.getLogger('opentrons').setLevel(logging.ERROR)


stats = Object()
stats.volume = 0



####################! FUNCTIONS - DO NOT MODIFY !######################### 
def washSamples(pipette, sourceSolutionWell, samples, volume, num_repeats=1):

    try:
        iter(samples)
        #print('samples are iterable')
    except TypeError:
        #print('samples arent iterable')
        samples = [samples]
    
    pipette.pick_up_tip()
    
    if(len(samples)==0):
        samples = [samples]
    print("Replacing solution on samples:" +str(samples) + " len=" + str(len(samples)))
    for i in range(0, num_repeats):
        print ("Iteration:"+ str(i))
        for s in samples:
            print ("Washing sample:" + str(s))
            pipette.aspirate(volume, sourceSolutionWell, rate=well_flow_rate)
            pipette.dispense(volume, s, rate=sample_flow_rate).blow_out()
            stats.volume += volume
    
    pipette.drop_tip()
    
def dilute_and_apply_fixative(pipette, sourceSolutionWell, dilutant_buffer_well, samples, volume):
    try:
        iter(samples)
        #print('samples are iterable')
    except TypeError:
        #print('samples arent iterable')
        samples = [samples]
    
    pipette.pick_up_tip()
    
    if(len(samples)==0):
        samples = [samples]
    print("Applying fixative to samples:" +str(samples) + " len=" + str(len(samples)))
 
    for s in samples:
        print ("Diluting fixative: " + str(s))
        pipette.aspirate(volume+50, dilutant_buffer_well, rate=well_flow_rate)
        pipette.dispense(volume+50, sourceSolutionWell, rate=well_flow_rate)
        for iterator in range(0, 3):
            print ("Mixing: " + str(iterator+1))
            pipette.aspirate(volume, sourceSolutionWell, rate=well_flow_rate)
            pipette.dispense(volume, sourceSolutionWell, rate=well_flow_rate)
        print ("Applying fixative to sample: " + str(s))
        pipette.aspirate(volume, sourceSolutionWell, rate=well_flow_rate)
        pipette.dispense(volume, s, rate=sample_flow_rate).blow_out()
        stats.volume += volume
    
    pipette.drop_tip()
    
def mix(pipette, sourceSolutionWell, volume, num_repeats):
   
    pipette.pick_up_tip()
    
    print("Mixing solution in samples:" +str(sourceSolutionWell))
    for i in range(0, num_repeats):
        print ("Iteration:"+ str(i))
        pipette.aspirate(volume, sourceSolutionWell, rate=2)
        pipette.dispense(volume, sourceSolutionWell, rate=2)
    
    pipette.drop_tip()


def countdown(t):
    while t:
        mins, secs = divmod(t, 60)
        timeformat = '{:02d}:{:02d}'.format(mins, secs)
        print(timeformat, end='\r')
        if (not debug): time.sleep(1)
        t -= 1
    print('off we go..')
    


In [3]:
###########################LABWARE SETUP#################################

protocol = execute.get_protocol_api(API_VERSION) 

if debug: print(protocol)

tiprack_300 = protocol.load_labware('opentrons_96_tiprack_300ul', labwarePositions.tiprack_300, "tiprack 300ul")

if debug: print(tiprack_300)

pipette_300 =  protocol.load_instrument('p300_single', 'right', tip_racks = [tiprack_300])
pipette_300.flow_rate.dispense = default_flow_rate
pipette_300.flow_rate.aspirate = default_flow_rate
pipette_300.starting_tip = tiprack_300.well(tiprack_starting_pos['tiprack_300'])

if debug: print(pipette_300)



par2 = protocol.load_labware('par2_plastic_wide_cslps', labwarePositions.par2, 'PAR2')


trough12 = protocol.load_labware('axygen_12well_trough', labwarePositions.buffers_reservoir, '12-trough buffers reservoir')


custom_96 = protocol.load_labware('black_96', labwarePositions.antibodies_plate, '96-well-plate')




if debug: print(par2)
    
buffer_wells = trough12.wells_by_name()

buffers = Object()
buffers.S1_PFA =  buffer_wells['A1']
buffers.S2 =  buffer_wells['A2']
buffers.PFA_S4 = buffer_wells['A3']
buffers.MeOH =  buffer_wells['A4']
buffers.PBS = buffer_wells['A5']
buffers.H2 = buffer_wells['A6']
buffers.pinkbuffer = buffer_wells['A7']
buffers.bluebuffer = buffer_wells['A8']
buffers.S4 =  buffer_wells['A9']

preblock_wells = custom_96.rows()[0]
antibody_wells = custom_96.rows()[1]
BS3_wells = custom_96.rows()[2]
rendering_wells = custom_96.rows()[3]

sample_chambers = []

for well in wellslist:
    sample_chambers.append(par2.wells_by_name()[well])

if debug: print(sample_chambers)


In [15]:
#################PROTOCOL####################
print("Starting the CODEX staining protocol")
protocol.home()

Starting the CODEX staining protocol


In [ ]:
#WASHING SAMPLES WITH PFA
print("first fix")
washSamples(pipette_300, buffers.PFA_S1, sample_chambers, wash_volume)
#INCUBATE FOR 10 MIN
print("first fix incubation: 10 min")
countdown(600)

In [ ]:
#WASHING SAMPLES WITH S2
print("washing in S2")
washSamples(pipette_300, buffers.S2, sample_chambers, wash_volume, num_repeats=2)

In [ ]:
#WASHING SAMPLES WITH PREBLOCK
print("preblocking")
for i in range (len(wellslist)):
    washSamples(pipette_300, preblock_wells[i], sample_chambers[i], wash_volume)
#INCUBATE 15 MIN
print("preblocking incubation: 15 min")
countdown(900)


In [ ]:
#APPLYING ANTIBODY COCKTAILS TO SAMPLES
print("applying antibodies")
for i in range (len(wellslist)):
    washSamples(pipette_300, antibody_wells[i], sample_chambers[i], wash_volume)

#INCUBATE 120 MIN
print("staining incubation: " + str(ABS_INCUBATION_TIME/3600) +"h" )
countdown(ABS_INCUBATION_TIME)

In [ ]:
#WASHING SAMPLES WITH S2
#three individual repeats below is because they need particular incubation time between them
print("washing with s2")
washSamples(pipette_300, buffers.S2, sample_chambers, wash_volume, num_repeats=2)
#INCUBATE FOR 10 MIN 
print("first incubation in s2: 5 min")
countdown(300)

In [ ]:
#WASHING SAMPLES WITH PFA
print("second fix")
washSamples(pipette_300, buffers.PFA_S4, sample_chambers, wash_volume)
#INCUBATE FOR 10 MIN 
print("second fix incubation: 5 min")
countdown(300)

In [ ]:
#WASHING SAMPLES WITH PBS
print("PBS wash")
washSamples(pipette_300, buffers.PBS, sample_chambers, wash_volume, num_repeats=2)

In [ ]:
#FIXING SAMPLES WITH Methanol
print("applying methanol")
washSamples(pipette_300, buffers.methanol, sample_chambers, wash_volume, num_repeats=2)
#INCUBATE FOR 5 MIN
print("methanol incubation: 5 min")
countdown(300)

In [ ]:
#WASHING SAMPLES WITH PBS
print("PBS wash")
washSamples(pipette_300, buffers.PBS, sample_chambers, wash_volume, num_repeats=2)

In [ ]:
#DILUTING AND APPLYING THE FIXATIVE
for i in range (len(wellslist)):
    dilute_and_apply_fixative(pipette_300, BS3_wells[i], buffers.PBS, sample_chambers[i], wash_volume)
    
print("third fix incubation: 10min")

countdown(600)

In [ ]:
#WASHING SAMPLES WITH PBS
washSamples(pipette_300, buffers.PBS, sample_chambers, wash_volume, 2)

In [ ]:
#PRE-CLEARING THE TISSUE
for i in range (3):
    washSamples(pipette_300, buffers.bluebuffer, sample_chambers, wash_volume, num_repeats=2)
    countdown(30)
    washSamples(pipette_300, buffers.pinkbuffer, sample_chambers, wash_volume, num_repeats=2)
    washSamples(pipette_300, buffers.H2, sample_chambers, wash_volume, num_repeats=2)

In [ ]:
#Equilibration in staining buffer
washSamples(pipette_300, buffers.pinkbuffer, sample_chambers, wash_volume)

In [ ]:
#RENDERING
for i in range (len(wellslist)):
    washSamples(pipette_300, rendering_wells[i], sample_chambers[i], wash_volume)
#INCUBATE FOR 10 MIN
countdown(600)

In [ ]:
#WASH SAMPLES IN PINK BUFFER
washSamples(pipette_300, buffers.pinkbuffer, sample_chambers, wash_volume, num_repeats=2)

In [ ]:
#STORAGE, washing samples every hour
for i in range (48):
    washSamples(pipette_300, buffers.S4,sample_chambers, wash_volume)
    countdown(3600)
    print("total dispensed volume: ", str(stats.volume))